#### Google Palm LLM & API key setup

In [1]:
import os
from langchain.llms import GooglePalm
import tqdm as notebook_tqdm
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
#initializing the llm
llm = GooglePalm(google_api_key=os.environ["GOOGLE_API_KEY"], temperature=0.2)

#### Connect with database and ask some basic questions

In [4]:
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

In [5]:
db_user = "root"
db_password = "root"
db_host = "localhost"
db_name = "zeho_tshirts"

db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}",sample_rows_in_table_info=3)

print(db.table_info)


CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), 
	CONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0 and 100))
)COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB

/*
3 rows from discounts table:
discount_id	t_shirt_id	pct_discount
1	1	10.00
2	2	15.00
3	3	20.00
*/


CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock

In [6]:
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True,return_direct=True)


In [7]:
qns1 = db_chain.run("How many t-shirts do we have left for nike in extra small size and white color?")

e:\LLM-Retail-Solution-Langchain-Google-Palm-Integration-for-Retail-Industry\Venv\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new SQLDatabaseChain chain...
How many t-shirts do we have left for nike in extra small size and white color?
SQLQuery:SELECT stock_quantity FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'
SQLResult: [(69,)]
> Finished chain.


Above is the correct answer 👍🏼

In [8]:
qns2 = db_chain.run("How much is the price of the inventory for all small size t-shirts?")



> Entering new SQLDatabaseChain chain...
How much is the price of the inventory for all small size t-shirts?
SQLQuery:SELECT SUM(price) FROM t_shirts WHERE size = 'S'
SQLResult: [(Decimal('436'),)]
> Finished chain.


It made a mistake here. The price is actually the price per unit but in real life database columns will not have perfect names. We need to tell it somehow that price is price per unit and the actual query should be,

SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'

In [9]:
qns2 = db_chain.run("SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'")



> Entering new SQLDatabaseChain chain...
SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'
SQLQuery:SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'
SQLResult: [(Decimal('21883'),)]
> Finished chain.


we will use qns2 value later on in this notebook. So hold on for now and let's check another query

In [10]:
qns3 = db_chain.run("If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?")



> Entering new SQLDatabaseChain chain...
If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?
SQLQuery:SELECT SUM(price - price * pct_discount) FROM discounts JOIN t_shirts ON discounts.t_shirt_id = t_shirts.t_shirt_id WHERE brand = 'Levi' AND CURDATE() BETWEEN discounts.start_date AND discounts.end_date

OperationalError: (pymysql.err.OperationalError) (1054, "Unknown column 'discounts.start_date' in 'where clause'")
[SQL: SELECT SUM(price - price * pct_discount) FROM discounts JOIN t_shirts ON discounts.t_shirt_id = t_shirts.t_shirt_id WHERE brand = 'Levi' AND CURDATE() BETWEEN discounts.start_date AND discounts.end_date]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

Above, it returned a wrong query which generated an error during query execution. It thinks discount
table would have start and end date which is normally true but in our table there is no start or end date column.
One thing we can do here is run the query directly.

In [12]:
sql_code = """
select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """

qns3 = db_chain.run(sql_code)



> Entering new SQLDatabaseChain chain...

select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 
SQLQuery:select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
SQLResult: [(Decimal('27578.900000'),)]
> Finished chain.



Now this is not much interesting because what is the point of giving it the ready made query? Well, we will use this same query later on for few shot learning

In [13]:
qns4 = db_chain.run("SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'")



> Entering new SQLDatabaseChain chain...
SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLQuery:SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLResult: [(Decimal('29154'),)]
> Finished chain.


In [14]:
qns5 = db_chain.run("How many white color Levi's t shirts we have available?")



> Entering new SQLDatabaseChain chain...
How many white color Levi's t shirts we have available?
SQLQuery:SELECT stock_quantity FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(42,), (91,), (10,)]
> Finished chain.


Once again above is the wrong answer. We need to use SUM(stock_quantity). Let's run the query explicitly. We will use the result of this query later on in the notebook

In [15]:
qns5 = db_chain.run("SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'")



> Entering new SQLDatabaseChain chain...
SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLQuery:SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(Decimal('143'),)]
> Finished chain.


#### Few shot learning

We will use few shot learning to fix issues we have seen so far

In [24]:
few_shots = [
    {'Question' : "How many t-shirts do we have left for Nike in XS size and white color?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
     'SQLResult': "Result of the SQL query",
     'Answer' : qns1},
    {'Question': "How much is the total price of the inventory for all S-size t-shirts?",
     'SQLQuery':"SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'",
     'SQLResult': "Result of the SQL query",
     'Answer': qns2},
    {'Question': "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?" ,
     'SQLQuery' : """SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """,
     'SQLResult': "Result of the SQL query",
     'Answer': qns3} ,
     {'Question' : "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?" ,
      'SQLQuery': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
      'SQLResult': "Result of the SQL query",
      'Answer' : qns4},
    {'Question': "How many white color Levi's shirt I have?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
     'SQLResult': "Result of the SQL query",
     'Answer' : qns5
     }
]

### Creating Semantic Similarity Based example selector

- create embedding on the few_shots
- Store the embeddings in a vector store: FAISS index
- Retrieve the the top most Semantically close example from the vector store

In [25]:
#Using Google 
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model='models/embedding-001')

In [26]:
to_vectorize = [" ".join(example.values()) for example in few_shots]

In [27]:
e = embeddings.embed_query("How many white color Levi's shirt I have?")
e[:5]

[0.017107653, -0.024187783, -0.026427852, -0.03232187, 0.04404449]

In [28]:
to_vectorize

["How many t-shirts do we have left for Nike in XS size and white color? SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS' Result of the SQL query [(69,)]",
 "How much is the total price of the inventory for all S-size t-shirts? SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S' Result of the SQL query [(Decimal('21883'),)]",
 "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)? SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from\n(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'\ngroup by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id\n  Result of the SQL query [(Decimal('27578.900000'),)]",
 "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount? SELECT SUM(price * stock_qua

In [29]:
#Using the FAISS vector database
from langchain.vectorstores import FAISS #vector embeddngs

#generating a vector store: 
vector_store = FAISS.from_texts(to_vectorize, embeddings, metadatas=few_shots)

In [30]:
#Checking sematic similarity: # Helping to pull similar looking queries
from langchain.prompts import SemanticSimilarityExampleSelector

example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vector_store,
    k=2,#number of examples
)

example_selector.select_examples({"Question": "How many Adidas T shirts I have left in my store?"})

[{'Question': 'How many t-shirts do we have left for Nike in XS size and white color?',
  'SQLQuery': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
  'SQLResult': 'Result of the SQL query',
  'Answer': '[(69,)]'},
 {'Question': "How many white color Levi's shirt I have?",
  'SQLQuery': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
  'SQLResult': 'Result of the SQL query',
  'Answer': "[(Decimal('143'),)]"}]

In [31]:
### my sql based instruction prompt
mysql_prompt = """You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: Query to run with no pre-amble
SQLResult: Result of the SQLQuery
Answer: Final answer here

No pre-amble.
"""

In [32]:
from langchain.prompts import FewShotPromptTemplate
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt

print(PROMPT_SUFFIX)

Only use the following tables:
{table_info}

Question: {input}


### Setting up PromptTemplete using input variables

In [33]:
from langchain.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["Question", "SQLQuery", "SQLResult","Answer",],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}",
)

In [34]:
print(_mysql_prompt)

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of

In [35]:
few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=mysql_prompt,
    suffix=PROMPT_SUFFIX,
    input_variables=["input", "table_info", "top_k"], #These variables are used in the prefix and suffix
)

In [36]:
new_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, prompt=few_shot_prompt, return_direct=True,return_intermediate_steps=True)

In [37]:
new_chain("How many white color Levi's shirt I have?")

e:\LLM-Retail-Solution-Langchain-Google-Palm-Integration-for-Retail-Industry\Venv\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new SQLDatabaseChain chain...
How many white color Levi's shirt I have?
SQLQuery:SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(Decimal('143'),)]
> Finished chain.


{'query': "How many white color Levi's shirt I have?",
 'result': "[(Decimal('143'),)]",
 'intermediate_steps': [{'input': "How many white color Levi's shirt I have?\nSQLQuery:",
   'top_k': '5',
   'dialect': 'mysql',
   'table_info': "\nCREATE TABLE discounts (\n\tdiscount_id INTEGER NOT NULL AUTO_INCREMENT, \n\tt_shirt_id INTEGER NOT NULL, \n\tpct_discount DECIMAL(5, 2), \n\tPRIMARY KEY (discount_id), \n\tCONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), \n\tCONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0 and 100))\n)COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB\n\n/*\n3 rows from discounts table:\ndiscount_id\tt_shirt_id\tpct_discount\n1\t1\t10.00\n2\t2\t15.00\n3\t3\t20.00\n*/\n\n\nCREATE TABLE t_shirts (\n\tt_shirt_id INTEGER NOT NULL AUTO_INCREMENT, \n\tbrand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, \n\tcolor ENUM('Red','Blue','Black','White') NOT NULL, \n\tsize ENUM('XS','S','M','L','XL') NOT NULL, \n\

Now this is working ok. Previously for this same question it was giving wrong answer because it did not use SUM clause around stock_quantity column

In [38]:
new_chain("How much is the price of the inventory for all small size t-shirts?")



> Entering new SQLDatabaseChain chain...
How much is the price of the inventory for all small size t-shirts?
SQLQuery:SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'
SQLResult: [(Decimal('21883'),)]
> Finished chain.


{'query': 'How much is the price of the inventory for all small size t-shirts?',
 'result': "[(Decimal('21883'),)]",
 'intermediate_steps': [{'input': 'How much is the price of the inventory for all small size t-shirts?\nSQLQuery:',
   'top_k': '5',
   'dialect': 'mysql',
   'table_info': "\nCREATE TABLE discounts (\n\tdiscount_id INTEGER NOT NULL AUTO_INCREMENT, \n\tt_shirt_id INTEGER NOT NULL, \n\tpct_discount DECIMAL(5, 2), \n\tPRIMARY KEY (discount_id), \n\tCONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), \n\tCONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0 and 100))\n)COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB\n\n/*\n3 rows from discounts table:\ndiscount_id\tt_shirt_id\tpct_discount\n1\t1\t10.00\n2\t2\t15.00\n3\t3\t20.00\n*/\n\n\nCREATE TABLE t_shirts (\n\tt_shirt_id INTEGER NOT NULL AUTO_INCREMENT, \n\tbrand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, \n\tcolor ENUM('Red','Blue','Black','White') NOT NU

In [39]:
new_chain("How much is the price of all white color levi t shirts?")



> Entering new SQLDatabaseChain chain...
How much is the price of all white color levi t shirts?
SQLQuery:SELECT SUM(price*stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(Decimal('6020'),)]
> Finished chain.


{'query': 'How much is the price of all white color levi t shirts?',
 'result': "[(Decimal('6020'),)]",
 'intermediate_steps': [{'input': 'How much is the price of all white color levi t shirts?\nSQLQuery:',
   'top_k': '5',
   'dialect': 'mysql',
   'table_info': "\nCREATE TABLE discounts (\n\tdiscount_id INTEGER NOT NULL AUTO_INCREMENT, \n\tt_shirt_id INTEGER NOT NULL, \n\tpct_discount DECIMAL(5, 2), \n\tPRIMARY KEY (discount_id), \n\tCONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), \n\tCONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0 and 100))\n)COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB\n\n/*\n3 rows from discounts table:\ndiscount_id\tt_shirt_id\tpct_discount\n1\t1\t10.00\n2\t2\t15.00\n3\t3\t20.00\n*/\n\n\nCREATE TABLE t_shirts (\n\tt_shirt_id INTEGER NOT NULL AUTO_INCREMENT, \n\tbrand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, \n\tcolor ENUM('Red','Blue','Black','White') NOT NULL, \n\tsize ENUM('XS','S

In [40]:
new_chain("If we have to sell all the Nike’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?")



> Entering new SQLDatabaseChain chain...
If we have to sell all the Nike’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?
SQLQuery:SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Nike'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
SQLResult: [(Decimal('21895.600000'),)]
> Finished chain.


{'query': 'If we have to sell all the Nike’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?',
 'result': "[(Decimal('21895.600000'),)]",
 'intermediate_steps': [{'input': 'If we have to sell all the Nike’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?\nSQLQuery:',
   'top_k': '5',
   'dialect': 'mysql',
   'table_info': "\nCREATE TABLE discounts (\n\tdiscount_id INTEGER NOT NULL AUTO_INCREMENT, \n\tt_shirt_id INTEGER NOT NULL, \n\tpct_discount DECIMAL(5, 2), \n\tPRIMARY KEY (discount_id), \n\tCONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), \n\tCONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0 and 100))\n)COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB\n\n/*\n3 rows from discounts table:\ndiscount_id\tt_shirt_id\tpct_discount\n1\t1\t10.00\n2\t2\t15.00\n3\t3\t20.00\n*/\n\n\nCREATE TABLE t_shirts (\n\tt_shirt_id INTEGER N

In [41]:
new_chain("If we have to sell all the Van Heuson T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?")



> Entering new SQLDatabaseChain chain...
If we have to sell all the Van Heuson T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?
SQLQuery:SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Van Huesen'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
SQLResult: [(Decimal('13176.400000'),)]
> Finished chain.


{'query': 'If we have to sell all the Van Heuson T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?',
 'result': "[(Decimal('13176.400000'),)]",
 'intermediate_steps': [{'input': 'If we have to sell all the Van Heuson T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?\nSQLQuery:',
   'top_k': '5',
   'dialect': 'mysql',
   'table_info': "\nCREATE TABLE discounts (\n\tdiscount_id INTEGER NOT NULL AUTO_INCREMENT, \n\tt_shirt_id INTEGER NOT NULL, \n\tpct_discount DECIMAL(5, 2), \n\tPRIMARY KEY (discount_id), \n\tCONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), \n\tCONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0 and 100))\n)COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB\n\n/*\n3 rows from discounts table:\ndiscount_id\tt_shirt_id\tpct_discount\n1\t1\t10.00\n2\t2\t15.00\n3\t3\t20.00\n*/\n\n\nCREATE TABLE t_shirts (\n\tt_shirt_id I

In [43]:
new_chain('How much revenue  our store will generate by selling all Van Heuson TShirts without discount?')



> Entering new SQLDatabaseChain chain...
How much revenue  our store will generate by selling all Van Heuson TShirts without discount?
SQLQuery:SELECT sum(price*stock_quantity) FROM t_shirts WHERE brand = 'Van Huesen'
SQLResult: [(Decimal('16528'),)]
> Finished chain.


{'query': 'How much revenue  our store will generate by selling all Van Heuson TShirts without discount?',
 'result': "[(Decimal('16528'),)]",
 'intermediate_steps': [{'input': 'How much revenue  our store will generate by selling all Van Heuson TShirts without discount?\nSQLQuery:',
   'top_k': '5',
   'dialect': 'mysql',
   'table_info': "\nCREATE TABLE discounts (\n\tdiscount_id INTEGER NOT NULL AUTO_INCREMENT, \n\tt_shirt_id INTEGER NOT NULL, \n\tpct_discount DECIMAL(5, 2), \n\tPRIMARY KEY (discount_id), \n\tCONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), \n\tCONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0 and 100))\n)COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB\n\n/*\n3 rows from discounts table:\ndiscount_id\tt_shirt_id\tpct_discount\n1\t1\t10.00\n2\t2\t15.00\n3\t3\t20.00\n*/\n\n\nCREATE TABLE t_shirts (\n\tt_shirt_id INTEGER NOT NULL AUTO_INCREMENT, \n\tbrand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL,